https://dacon.io/competitions/official/235697/codeshare/2429?page=1&dtype=recent

https://dacon.io/competitions/official/235957/codeshare/6008?page=1&dtype=recent

# **✨Run**

In [ ]:
# TRAIN & VALIDATION
%cd /content/drive/MyDrive/DACON/Image_Classification/Landmark
!python train.py

/content/drive/MyDrive/DACON/Image_Classification/Landmark
Device: cuda:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
[info msg] arguments

image_path  :  /content/drive/MyDrive/DACON/Image_Classification/Landmark/data/train/
label_path  :  /content/drive/MyDrive/DACON/Image_Classification/Landmark/data/train.csv
kfold_idx  :  1
epochs  :  20
batch_size  :  32
lr  :  0.001
patience  :  8
seed  :  777
worker  :  4
model  :  resnet50
pretrained  :  False
resume  :  None
device  :  cuda:0
comments  :  None
train_aug  :  Compose([
  Resize(always_apply=False, p=1, height=224, width=224, interpolation=1),
  VerticalFlip(always_apply=False, p=0.2),
  HorizontalFlip(always_apply=False, p=0.2),
  ShiftScaleRotate(always_apply=False, p=0.5, shift_limit_x=(-0.1, 0.1), shift_limit_y=(-0.1, 0.1), scale_limit=(-0.15000000000000002, 0.1499999999999999), rotate_limit=(-60, 60), interpolation=1, border_mode=4, value=None, mask_value=None, rotate_method='largest_box'),
  Ran

In [ ]:
# INFERENCE
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
%cd /content/drive/MyDrive/DACON/Image_Classification/Landmark

!python inference.py --image_path ./data/test/ \
--label_path ./data/sample_submission.csv \
--weight_path ./save/effb4-f2-e20/model_best.pth \
--out_path ./out/ \
--model 'resnet50' \
--batch_size 8 \
--device {device}

/content/drive/MyDrive/DACON/Image_Classification/Landmark
[info msg] arguments
image_path : ./data/test/
label_path : ./data/sample_submission.csv
weight_path : ./save/effb4-f2-e20/model_best.pth
out_path : ./out/
model : efficientnet_b4
batch_size : 4
device : cuda:0
[info msg] weight ./save/effb4-f2-e20/model_best.pth is loaded
[Info msg] model efficientnet_b4 is loaded
[Info msg] INFERENCE Start !!!
100% 50/50 [00:10<00:00,  4.71it/s]
[info msg] submission fils is saved to ./out/08231024_ensemble_submission.csv


# **🐮 Libraries**

In [ ]:
%%time

!pip install --upgrade albumentations -qqq
!pip install timm -qqq
!pip install ttach -qqq

     |████████████████████████████████| 509 kB 13.8 MB/s 
CPU times: user 63.7 ms, sys: 38.2 ms, total: 102 ms
Wall time: 11.4 s


In [ ]:
import os
import cv2
import sys
from tqdm import tqdm
import random
import easydict

import pandas as pd
from glob import glob
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

# **🥽 Configuration**

In [ ]:
from albumentations.augmentations.geometric.resize import Resize
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

args = easydict.EasyDict({ # EasyDict 는 속성으로 dict 값에 액세스 할 수 있습니다.(재귀적으로 작동). 
    'image_path' : '/content/drive/MyDrive/DACON/Image_Classification/Landmark/data/train/',
    'label_path' : '/content/drive/MyDrive/DACON/Image_Classification/Landmark/data/train.csv',
    'kfold_idx' : 0 , # 0~4까지 바꿔가면서 5번 학습

    'epochs' : 10,
    'batch_size' : 16,
    'lr' : 1e-3,
    'patience' : 8,
    'seed' : 777,
    'worker' : torch.cuda.device_count() * 4,
    'model' : 'efficientnet-b0',

    'resume' : None,
    'device' : device,
    'comments': None,




    # augmentation
    'train_aug' : A.Compose([
                         A.Resize(224, 224),
                         A.VerticalFlip(p = 0.5),
                         A.HorizontalFlip(p=0.5),
                         A.ShiftScaleRotate(shift_limit=0.1, 
                           scale_limit=0.15, 
                           rotate_limit=60, 
                           p=0.5),
                        #  A.HueSaturationValue(
                                # hue_shift_limit=0.2, 
                                # sat_shift_limit=0.2, 
                                # val_shift_limit=0.2, 
                                # p=0.5
                            # ),
                        #  A.RandomBrightnessContrast(
                                # brightness_limit=(-0.1,0.1), 
                                # contrast_limit=(-0.1, 0.1), 
                                # p=0.5
                            # ),
                         A.Normalize(max_pixel_value=1.0, p=1),
                         ToTensorV2() # h,w,c -> c,h,w
  ]),
    'val_aug' : A.Compose([
                        A.Resize(224,224),
                        A.Normalize(max_pixel_value=1.0, p=1),
                        ToTensorV2()
    ]),

    'test_aug' : A.Compose([
                        #  A.Resize(780, 780),
                        #  A.VerticalFlip(p=0.5),
                        #  A.HorizontalFlip(p=0.5),
                        A.Resize(224,224),
                        A.Normalize(max_pixel_value=1.0, p=1),
                        ToTensorV2()
  ])
})

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(args.seed)

In [ ]:
#GPU 체크 및 할당
if torch.cuda.is_available():    
    #device = torch.device("cuda:0")
    print('Device:', device)
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

Device: cuda:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


# **🍿 Data Preprocessing**

### **CustomDataset**

In [ ]:
class DatasetLM(Dataset):
    def __init__(self, image_folder, label_df, transforms):
        self.image_folder = image_folder
        self.label_df = label_df
        self.transforms = transforms

    def __len__(self):
        return len(self.label_df)

    def __getitem__(self, index):
        image_fn = self.image_folder + str(self.label_df.iloc[index,0])

        image = cv2.imread(image_fn)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


        label = self.label_df.iloc[index,1]

        if self.transforms:
            image = self.transforms(image=image)['image']
        
        return image, label

In [ ]:
# train_set = DatasetLM(
#     image_folder = args.image_path,
#     label_df = train_data,
#     transforms=args.train_aug
# )

# print(next(iter(train_set[0])).shape)
# type(next(iter(train_set[0])))


torch.Size([3, 224, 224])


torch.Tensor

# **🧦 Train & Validation**

In [ ]:
# train

def train(train_loader, model, loss_func, device, optimizer, scheduler=None):
    n = 0
    running_loss = 0.0
    running_corrects = 0

    epoch_loss = 0.0
    epoch_acc = 0.0

    model.train()

    # sys.stdout : 보통 프로그램 수행 과정에서 몇 시에 어떤 작업을 어떤 식으로 수행하고 있으며 그 결과는 어떠한지 등의 정보를 가지는 로그를 남길 때 stdout 은 일반적인 내용을, stderr 는 에러 발생 시 관련 내용을 출력하기 위해 사용할 수 있습니다.
    with tqdm(train_loader, total=len(train_loader), desc='Train', file=sys.stdout) as iterator:
        for train_x, train_y in iterator:
            train_x, train_y = train_x.to(device), train_y.to(device)

            output = model(train_x)
            _, preds = torch.max(output,1)

            loss = loss_func(output, train_y)

            n += train_x.size(0)
            running_loss += loss.item() * train_x.size(0)
            running_corrects += (preds==train_y).sum().item()

            
            epoch_acc = running_corrects /  n
            epoch_loss = running_loss / n

            log = f"Train_loss - {epoch_loss:.5f}, Train_acc - {epoch_acc:.5f}"

            iterator.set_postfix_str(log)

            optimizer.zero_grad() #배치마다 optimizer 초기화
            loss.backward() #손실함수 기준 역전파 
            torch.nn.utils.clip_grad_norm(model.parameters(),1) # Clips gradient norm of an iterable of parameters.
            optimizer.step() #가중치 최적화
        
    if scheduler:
        scheduler.step(epoch_loss)
    
    return epoch_loss, epoch_acc


def validate(valid_loader, model, loss_func, device, scheduler=None):

    n = 0
    running_loss = 0.0
    running_corrects = 0

    epoch_loss = 0.0
    epoch_acc = 0.0

    model.eval()

    with tqdm(valid_loader, total=len(valid_loader), desc='Valid', file=sys.stdout) as iterator:
        for x,y in iterator:
            x,y = x.to(device), y.to(device)

            with torch.no_grad():
                output = model(x)

            loss = loss_func(output,y)

            n += x.size(0)
            running_loss += loss.item() * x.size(0)

            epoch_loss = running_loss / n

            _, preds = torch.max(output,1)
            running_corrects += (preds==y).sum().item()
            epoch_acc = running_corrects / n

            log = f"Val_loss - {epoch_loss}, Val_acc - {epoch_acc}"

            iterator.set_postfix_str(log)
    if scheduler:
        scheduler.step(epoch_loss)

    return epoch_loss, epoch_acc

In [ ]:
# checkin args
print('=' * 75)
print('[info msg] arguments\n')
for key, value in vars(args).items(): # 객체의 __dict__ 속성 반환
    print(key, ' : ', value)
print('\n','=' * 75)

assert os.path.isdir(args.image_path), 'wrong path'
assert os.path.isfile(args.label_path), 'wrong path'
if (args.resume):
    assert os.path.isfile(args.resume), 'wrong path'
assert args.kfold_idx < 5


# Data Split for Fold & DataLoader Generation
data_set = pd.read_csv(args.label_path)
valid_idx_nb = int(len(data_set) * (1/5)) # int(723/5)
valid_idx = np.arange(valid_idx_nb * args.kfold_idx, valid_idx_nb * (args.kfold_idx + 1))

print('[info msg] validation fold idx !!\n')        
print(valid_idx)
print('=' * 75)

train_data = data_set.drop(valid_idx) # valid_idx에 해당되는 데이터를 제거하고 남은 데이터들
valid_data = data_set.iloc[valid_idx] # valid_idx에 포함되는 데이터들

train_set = DatasetLM(
    image_folder = args.image_path,
    label_df = train_data,
    transforms=args.train_aug
)


valid_set = DatasetLM(
    image_folder = args.image_path,
    label_df = valid_data,
    transforms = args.val_aug
)

train_data_loader = DataLoader(
    train_set,
    batch_size = args.batch_size,
    shuffle=True)

valid_data_loader = DataLoader(
    valid_set,
    batch_size = args.batch_size,
    shuffle=False
)

model = None


if(args.resume):
    timm.create_model('efficientnet_b0', checkpoint_path=args.resume)
    print('[info msg] pre-trained weight is loaded !! \n')
    print(args.resume)
    print('=' * 70)
else:
    print(f'[info msg] {args.model} model is created\n')
    model = timm.create_model('efficientnet_b0', pretrained=False, num_classes=10,\
                          drop_rate=0.3)
    print('=' * 70)

if args.device == 'cuda:0' and torch.cuda.device_count() > 1 :
    model = torch.nn.DataParallel(model)

model.to(args.device)

optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
criterion = torch.nn.CrossEntropyLoss() # softmax automatically
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer = optimizer,
    mode='min',
    patience=2,
    factor=0.3,
    verbose=True
)


train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

best_loss = float('inf')

patience = 0

date_time = datetime.now().strftime('%m%d%H%M%S')
if not os.path.exists('./save'):
    os.mkdir('./save')
SAVE_DIR = os.path.join('./save', date_time)

print('[info msg] TRAINING START !!!!\n')
startTime = datetime.now()

for epoch in range(args.epochs):
    print(f"Epoch {epoch+1}/{args.epochs}")
    train_epoch_loss, train_epoch_acc = train(
        train_loader = train_data_loader,
        model=model,
        loss_func = criterion,
        device=args.device,
        optimizer=optimizer,
    )
    # list appending
    train_loss.append(train_epoch_loss)
    train_acc.append(train_epoch_acc)

    valid_epoch_loss, valid_epoch_acc = validate(
        valid_loader = valid_data_loader,
        model=model,
        loss_func=criterion,
        device=args.device,
        scheduler=scheduler,
    )
    # list appending
    valid_loss.append(valid_epoch_loss)
    valid_acc.append(valid_epoch_acc)


    # Save Best Model
    if best_loss > valid_epoch_loss:
        patience = 0
        best_loss = valid_epoch_loss

        Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(SAVE_DIR,'model_best.pth'))
        print(f"MODEL IS SAVED TO {date_time}!!!!")

    else :
        patience += 1
        if patience > args.patience -1:
            print("=========" * 10)
            print('[Info message] Early Stopper is activated')
            break

elapsed_time = datetime.now() - startTime

train_loss = np.array(train_loss)
train_acc = np.array(train_acc)
valid_loss = np.array(valid_loss)
valid_acc = np.array(valid_acc)

best_loss_pos = np.argmin(valid_loss) # index 출력


print('=' * 70)
print('[info msg] TRAINING is DONE\n')
print(f"Time taken: {elapsed_time}.")
print(f"best loss is {best_loss} w/ acc {valid_acc[best_loss_pos]} at epoch : {best_loss_pos}.")    

print('=' * 70)
print(f'[info msg] {args.model} model weight and log are saved to {SAVE_DIR}\n')

# Save Log
with open(os.path.join(SAVE_DIR, 'log.txt'),'w') as f:
    for key, value in vars(args).items():
        f.write(f"{key} : {value}\n")

    f.write('\n')
    f.write(f"Total Epochs : {str(train_loss.shape[0])}\n")
    f.write(f"Time Taken : {str(elapsed_time)}")
    f.write(f"Best_Train_Loss {np.min(train_loss)} w/acc {np.argmin(train_loss)} at epoch : {np.argmin(train_loss)}")
    f.write(f'Best_Valid_Loss {np.min(valid_loss)} w/ acc {valid_acc[np.argmin(valid_loss)]} at epoch : {np.argmin(valid_loss)}')

plt.figure(figsize=(15,5))
plt.subplot(1,2,1) # nrows, ncols, index
plt.plot(train_loss, label='train_loss')
plt.plot(valid_loss,'o',label='valid_loss')
plt.axvline(x=best_loss_pos, color='r', linestyle='--', linewidth=1.5)
plt.legend()

plt.subplot(1,2,2)
plt.plot(train_acc, label='train_acc')
plt.plot(valid_acc, 'o', label='valid_acc')
plt.axvline(x=best_loss_pos, color='r', linestyle='--', linewidth=1.5)
plt.legend()
plt.savefig(os.path.join(SAVE_DIR, 'history.png'))

# **🍯 Inference**


In [ ]:
%%time

!pip install --upgrade albumentations -qqq
!pip install timm -qqq
!pip install ttach -qqq

import cv2
import copy
import torch
import argparse
import ttach as tta
from tqdm import tqdm
from glob import glob
import numpy as np
import pandas as pd
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

import timm
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset

CPU times: user 1.22 s, sys: 179 ms, total: 1.4 s
Wall time: 9.59 s


In [ ]:
class DatasetLMT(Dataset):
    def __init__(self, image_folder, label_df, transforms):
        self.image_folder = image_folder
        self.label_df = label_df
        self.transforms = transforms

    def __len__(self):
        return len(self.label_df)

    def __getitem__(self, index):
        image_fn = self.image_folder + str(self.label_df.iloc[index,0])

        image = cv2.imread(image_fn)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


        label = self.label_df.iloc[index,1]

        if self.transforms:
            image = self.transforms(image=image)['image'] / 255.0
        
        return image, label

In [ ]:
base_transforms = A.Compose([        
        ToTensorV2(),
        ])

test_df = pd.read_csv('/content/drive/MyDrive/DACON/Image_Classification/Landmark/data/sample_submission.csv')

test_set = DatasetLMT(
        image_folder = args.image_path,
        label_df = test_df,
        transforms = base_transforms
    )    

In [ ]:
# with torch.inference_mode():

#     for i, (imgs, _) in enumerate(tqdm(test_data_loader)):
#         imgs = imgs.to(device)
#         output = tta_model(imgs)
#         _, preds = torch.max(output,1)
#         print('\n',output)
#         print('Torch.max한 것 \n ----->', torch.max(output,1),'\n')
#         print(f'Torch.max의 index \n -----> {preds}','\n')
#         print(f'preds 값 추출 \n ------> {preds.long().cpu().detach().cpu().numpy()[:,np.newaxis]}, type : {type(preds.long().cpu().detach().cpu().numpy()[:,np.newaxis])}')
#         print('DETACH 한 것 \n ---->' ,output.detach().cpu().numpy().squeeze(),'\n')
#         print('np.argmax 한 것 \n ---->',np.argmax(output.detach().cpu().numpy().squeeze(),1), '\n\n\n')

  4%|▍         | 1/25 [00:00<00:10,  2.33it/s]


 tensor([[-1.4060, -3.7310, -3.8231, 11.7666,  0.2489,  5.9337, -1.6594, -3.5091,
         -5.1482, -2.9927],
        [-1.4059, -3.7280, -3.8231, 11.7647,  0.2474,  5.9341, -1.6587, -3.5080,
         -5.1486, -2.9935],
        [-1.4063, -3.7230, -3.8230, 11.7620,  0.2444,  5.9358, -1.6570, -3.5062,
         -5.1498, -2.9950],
        [-1.4060, -3.7264, -3.8231, 11.7643,  0.2467,  5.9346, -1.6581, -3.5077,
         -5.1492, -2.9943],
        [-1.4061, -3.7267, -3.8232, 11.7638,  0.2465,  5.9349, -1.6583, -3.5072,
         -5.1488, -2.9936],
        [-1.4061, -3.7298, -3.8234, 11.7656,  0.2482,  5.9342, -1.6593, -3.5083,
         -5.1483, -2.9926],
        [-1.4061, -3.7309, -3.8235, 11.7662,  0.2487,  5.9342, -1.6596, -3.5086,
         -5.1481, -2.9921],
        [-1.4055, -3.7328, -3.8234, 11.7679,  0.2506,  5.9324, -1.6603, -3.5101,
         -5.1478, -2.9924]], device='cuda:0')
Torch.max한 것 
 -----> torch.return_types.max(
values=tensor([11.7666, 11.7647, 11.7620, 11.7643, 11.7638, 11

  8%|▊         | 2/25 [00:00<00:09,  2.31it/s]


 tensor([[-1.4054, -3.7264, -3.8231, 11.7657,  0.2479,  5.9333, -1.6582, -3.5089,
         -5.1497, -2.9955],
        [-1.4055, -3.7236, -3.8229, 11.7642,  0.2463,  5.9339, -1.6572, -3.5080,
         -5.1503, -2.9965],
        [-1.4056, -3.7309, -3.8232, 11.7672,  0.2495,  5.9329, -1.6595, -3.5096,
         -5.1484, -2.9934],
        [-1.4056, -3.7306, -3.8233, 11.7670,  0.2494,  5.9331, -1.6596, -3.5095,
         -5.1484, -2.9932],
        [-1.4057, -3.7255, -3.8227, 11.7632,  0.2462,  5.9340, -1.6577, -3.5073,
         -5.1489, -2.9946],
        [-1.4063, -3.7268, -3.8229, 11.7629,  0.2459,  5.9350, -1.6582, -3.5067,
         -5.1484, -2.9931],
        [-1.4063, -3.7283, -3.8229, 11.7635,  0.2465,  5.9349, -1.6586, -3.5070,
         -5.1480, -2.9925],
        [-1.4062, -3.7256, -3.8230, 11.7633,  0.2458,  5.9351, -1.6578, -3.5070,
         -5.1492, -2.9942]], device='cuda:0')
Torch.max한 것 
 -----> torch.return_types.max(
values=tensor([11.7657, 11.7642, 11.7672, 11.7670, 11.7632, 11

 12%|█▏        | 3/25 [00:01<00:09,  2.30it/s]

...


 tensor([[-1.4060, -3.7275, -3.8230, 11.7644,  0.2471,  5.9343, -1.6584, -3.5077,
         -5.1487, -2.9936],
        [-1.4060, -3.7268, -3.8229, 11.7638,  0.2466,  5.9343, -1.6582, -3.5075,
         -5.1487, -2.9938],
        [-1.4061, -3.7284, -3.8232, 11.7647,  0.2474,  5.9345, -1.6588, -3.5078,
         -5.1486, -2.9931],
        [-1.4060, -3.7273, -3.8232, 11.7646,  0.2471,  5.9345, -1.6584, -3.5078,
         -5.1490, -2.9939],
        [-1.4061, -3.7266, -3.8231, 11.7639,  0.2465,  5.9348, -1.6582, -3.5074,
         -5.1490, -2.9938],
        [-1.4059, -3.7287, -3.8236, 11.7658,  0.2481,  5.9343, -1.6591, -3.5085,
         -5.1489, -2.9934],
        [-1.4062, -3.7294, -3.8233, 11.7650,  0.2477,  5.9346, -1.6591, -3.5079,
         -5.1482, -2.9925],
        [-1.4062, -3.7264, -3.8233, 11.7641,  0.2465,  5.9351, -1.6582, -3.5074,
         -5.1492, -2.9939]], device='cuda:0')
Torch.max한 것 
 -----> torch.return_types.max(
values=tensor([11.7644, 11.7638, 11.7647, 11.7646, 11.7639, 11

 96%|█████████▌| 24/25 [00:10<00:00,  2.27it/s]


 tensor([[-1.4055, -3.7296, -3.8236, 11.7673,  0.2495,  5.9332, -1.6595, -3.5096,
         -5.1490, -2.9939],
        [-1.4059, -3.7302, -3.8232, 11.7656,  0.2485,  5.9337, -1.6594, -3.5085,
         -5.1480, -2.9925],
        [-1.4057, -3.7292, -3.8227, 11.7646,  0.2479,  5.9331, -1.6588, -3.5082,
         -5.1478, -2.9931],
        [-1.4060, -3.7264, -3.8232, 11.7646,  0.2469,  5.9346, -1.6582, -3.5079,
         -5.1494, -2.9944],
        [-1.4062, -3.7298, -3.8233, 11.7651,  0.2478,  5.9345, -1.6593, -3.5080,
         -5.1481, -2.9923],
        [-1.4053, -3.7269, -3.8230, 11.7659,  0.2481,  5.9329, -1.6583, -3.5091,
         -5.1495, -2.9954],
        [-1.4062, -3.7290, -3.8233, 11.7647,  0.2475,  5.9346, -1.6591, -3.5077,
         -5.1483, -2.9926],
        [-1.4055, -3.7305, -3.8231, 11.7672,  0.2495,  5.9326, -1.6593, -3.5098,
         -5.1485, -2.9938]], device='cuda:0')
Torch.max한 것 
 -----> torch.return_types.max(
values=tensor([11.7673, 11.7656, 11.7646, 11.7646, 11.7651, 11

100%|██████████| 25/25 [00:10<00:00,  2.29it/s]


 tensor([[-1.4060, -3.7255, -3.8228, 11.7635,  0.2460,  5.9346, -1.6577, -3.5073,
         -5.1492, -2.9946],
        [-1.4060, -3.7272, -3.8230, 11.7643,  0.2469,  5.9343, -1.6584, -3.5077,
         -5.1488, -2.9938],
        [-1.4065, -3.7255, -3.8229, 11.7624,  0.2450,  5.9358, -1.6577, -3.5062,
         -5.1489, -2.9935],
        [-1.4054, -3.7254, -3.8231, 11.7658,  0.2477,  5.9333, -1.6578, -3.5090,
         -5.1502, -2.9963],
        [-1.4063, -3.7287, -3.8234, 11.7647,  0.2473,  5.9350, -1.6590, -3.5076,
         -5.1485, -2.9926],
        [-1.4061, -3.7313, -3.8235, 11.7661,  0.2488,  5.9341, -1.6598, -3.5085,
         -5.1478, -2.9919],
        [-1.4055, -3.7285, -3.8223, 11.7653,  0.2481,  5.9326, -1.6584, -3.5090,
         -5.1483, -2.9942]], device='cuda:0')
Torch.max한 것 
 -----> torch.return_types.max(
values=tensor([11.7635, 11.7643, 11.7624, 11.7658, 11.7647, 11.7661, 11.7653],
       device='cuda:0'),
indices=tensor([3, 3, 3, 3, 3, 3, 3], device='cuda:0')) 

Torch.max

In [ ]:
base_transforms = A.Compose([        
        ToTensorV2(),
        ])

tta_transforms = tta.Compose([
    tta.Rotate90([0,90,180,270]),
])

def main():
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    parser = argparse.ArgumentParser()
    parser.add_argument('--image_path',type=str, default='./data/')
    parser.add_argument('--label_path',type=str, default='./data/sample_submission.csv')
    parser.add_argument('--weight_path',type=str, default='./save/')
    parser.add_argument('--out_path',type=str,default='./output/')

    parser.add_argument('--model',type=str,default='efficientnet-b0')
    parser.add_argument('--batch_size', type=int,default=4)

    parser.add_argument('--device',type=str,default=device)

    args = parser.parse_args()

    assert os.path.isdir(args.image_path), 'Wrong Image Path'
    assert os.path.isfile(args.label_path), 'Wrong Label Path'
    assert os.path.isfile(args.weight_path), 'Wrong Weight Path'
    assert os.path.isdir(args.out_path), 'Wrong Out Path'

    print('=' * 70)
    print('[info msg] arguments')
    for key, value in vars(args).items():
        print(key, ":", value)

    weights = glob(os.path.join(args.weight_path))

    test_df = pd.read_csv(args.label_path)

    test_set = DatasetLMT(
        image_folder = args.image_path,
        label_df = test_df,
        transforms = base_transforms
    )    

    submission_df = copy.copy(test_df)


    for weight in weights:
        model = timm.create_model(args.model, pretrained=False, num_classes=10)
        model.load_state_dict(torch.load(weight, map_location=args.device))
        print('=' * 70)
        print(f'[info msg] weight {weight} is loaded')
        print(f'[Info msg] model {args.model} is loaded')

        test_data_loader = DataLoader(
            test_set,
            batch_size = args.batch_size,
            shuffle=False,
        )

        model.to(args.device)
        

        model.eval()
        tta_model = tta.ClassificationTTAWrapper(model, tta_transforms) # Initializes internal Module state, shared by both nn.Module and Script

        batch_size = args.batch_size
        batch_index = 0

        print('=' * 70)
        print('[Info msg] INFERENCE Start !!!')

        with torch.inference_mode():

            for i, (imgs, _) in enumerate(tqdm(test_data_loader)):
                imgs = imgs.to(args.device)
                outputs = tta_model(imgs)  # soft
                _, preds = torch.max(outputs,1)
                # outputs = (outputs > 0.5).astype(int) # hard vote
                batch_index = i * batch_size
                submission_df.iloc[batch_index : batch_index+batch_size, 1:] += preds.long().cpu().detach().numpy()[:,np.newaxis]

    ###
    # submission_df.iloc[:,1:] = (submission_df.iloc[:,1:] / len(weights) > 0.35)
    SAVE_FN = os.path.join(args.out_path, datetime.now().strftime('%m%d%H%M') + '_ensemble_submission.csv')

    submission_df.to_csv(
        SAVE_FN,
        index=False
    )

    print('=' * 70)
    print(f'[info msg] submission fils is saved to {SAVE_FN}')

if __name__ == '__main__':
    main()

# **🍢 Custom Inference**


In [ ]:
def inference(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():

        bar = tqdm(enumerate(test_loader), total=len(test_loader))

        for i, (img,label) in bar:
            img = img.to(device)

            output = model(img)
            # output = output.argmax(dim=1,keepdim=True).squeeze(1)
            _,pred = torch.max(output,1)

            model_pred.extend(pred.tolist())

    return model_pred

In [ ]:
class DatasetLMT(Dataset):
    def __init__(self, image_folder, label_df, transforms):
        self.image_folder = image_folder
        self.label_df = label_df
        self.transforms = transforms

    def __len__(self):
        return len(self.label_df)

    def __getitem__(self, index):
        image_fn = self.image_folder + str(self.label_df.iloc[index,0])

        image = cv2.imread(image_fn)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


        label = self.label_df.iloc[index,1]

        if self.transforms:
            image = self.transforms(image=image)['image'] / 255.0
        
        return image, label

In [ ]:
base_transforms = A.Compose([  
        A.Resize(224,224),
        # A.Normalize(max_pixel_value=1.0, p=1),
        ToTensorV2(),
        ])

# tta_transforms = tta.Compose([
#     tta.Rotate90([0,90,180,270]),
# ])

root_path = '/content/drive/MyDrive/DACON/Image_Classification/Landmark/'
test_df = pd.read_csv(root_path + 'data/sample_submission.csv')
train_df = pd.read_csv(root_path +'data/train.csv')
test_data_path = root_path + 'data/test/'
train_data_path = root_path + 'data/train/'
weight = root_path + 'save/0823105650/model_best.pth'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
batch_size = 1

train_dataset = DatasetLMT(
    image_folder = train_data_path,
    label_df = train_df,
    transforms = base_transforms
)

train_data_loader = DataLoader(
    train_dataset[0],
    batch_size=1,
    shuffle=False
)


test_dataset = DatasetLMT(
        image_folder = test_data_path,
        label_df = test_df,
        transforms = base_transforms
    )

test_data_loader = DataLoader(
        test_dataset,
            batch_size = batch_size,
            shuffle=False,
    )

submission_df = copy.copy(test_df)

model = timm.create_model('resnet50', pretrained=False, num_classes=10,checkpoint_path=weight)
# model.load_state_dict(torch.load(weight, map_location=args.device))

model.to(device)

preds = inference(model, test_data_loader, device)
preds

In [ ]:
print(len(train_dataset))
train_dataset[722]

723


(tensor([[[2.8344, 2.8173, 2.8858,  ..., 0.4198, 0.4027, 0.3856],
          [2.9885, 2.9714, 2.9372,  ..., 0.4369, 0.4369, 0.4027],
          [3.3481, 3.7249, 3.3824,  ..., 0.4883, 0.5397, 0.4369],
          ...,
          [1.0192, 1.5672, 1.5843,  ..., 2.3207, 2.1665, 1.9097],
          [0.7281, 0.7281, 0.7109,  ..., 1.4987, 2.4405, 2.0124],
          [0.6424, 0.7109, 0.8479,  ..., 2.0980, 1.4987, 1.9439]],
 
         [[3.0032, 3.0207, 3.0908,  ..., 0.8324, 0.8499, 0.9024],
          [3.0557, 3.0207, 3.0557,  ..., 0.7973, 0.8499, 0.8849],
          [3.3008, 3.6335, 3.3359,  ..., 0.8148, 0.8674, 0.8499],
          ...,
          [1.1825, 1.8127, 1.8478,  ..., 2.4255, 2.2504, 1.9528],
          [1.0249, 1.0249, 1.0074,  ..., 1.5326, 2.4955, 2.0403],
          [0.9724, 0.9549, 1.0599,  ..., 2.1454, 1.5151, 1.9703]],
 
         [[3.2870, 3.2522, 3.3742,  ..., 0.9690, 0.9690, 0.9690],
          [3.1999, 3.1825, 3.2522,  ..., 0.9515, 0.9864, 0.9690],
          [3.3045, 3.6531, 3.3916,  ...,

In [ ]:
weight = root_path + 'save/effb4-f2-e20/model_best.pth'
# model = timm.create_model('resnet50',num_classes=10,checkpoint_path=weight)
model = timm.create_model('efficientnet_b4', pretrained=False, num_classes=10)
model.load_state_dict(torch.load(weight, map_location=args.device))
model.to(device)
img = train_dataset[400][0] # Train 데이터 마지막까지 같은 클래스로 예측함.... 총 722개
img = img.to(device)
img = img.unsqueeze(0)
output = model(img)
output
# pred = torch.argmax(output, dim=1)
# _, pred = torch.max(output,1)
# print(pred)

tensor([[ 1.5065, -2.4709, -2.0307, -0.7362, -4.4597, -2.3862, -0.5020, -0.7906,
         -3.6704, -7.5762]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
train_df.iloc[3,0]

'004.PNG'